<a href="https://colab.research.google.com/github/Priyam-27/Machine-learning/blob/master/Churn_Modelling_Artificial_Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# New Section

####Test whether GPU is working or not

In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

#### Which GPU is working

In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

#### RAM Information

In [ ]:
! cat /proc/meminfo

#### CPU Information

In [ ]:
!cat /proc/cpuinfo

#### Mounting drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
pip install scikit-learn==0.21.2

#### Installing Libraries

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
pd.options.display.max_rows=None
pd.options.display.max_columns=None
sns.set_style('darkgrid')

In [ ]:
data=pd.read_csv('/content/drive/MyDrive/Churn_Modelling/Churn_Modelling.csv')
df=data.copy()
print(f'Shape of data is : {df.shape}')

In [ ]:
df.head()

In [ ]:
for i in df.columns:
  print(f'{i}    :    {df[i].nunique()} values')

Since RowNumber and Customerid are unique for all cases so we can drop them and Surname has too many categories so it also won't have that much impact over target

In [ ]:
X=df.iloc[:,3:13]
Y=df.iloc[:,-1]

In [ ]:
X.head()

In [ ]:
Y.head()

In [ ]:
X_cat=[i for i in X.columns if df[i].dtypes == 'O']
X_cat

In [ ]:
X_num=[i for i in X.columns if i not in X_cat]
X_num

In [ ]:
X_cat_enc=pd.get_dummies(df[X_cat], drop_first=True)
X_cat_enc.head()

In [ ]:
X=pd.concat([X_cat_enc, X[X_num]], axis=1)
X.head()

Splitting into train test set

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train,x_test,y_train,y_test =train_test_split(X,Y,test_size=0.3, random_state=0)

In [ ]:
print(f'Shape of x_train : {x_train.shape}')
print(f'Shape of x_test : {x_test.shape}')
print(f'Shape of y_train : {y_train.shape}')
print(f'Shape of y_test : {y_test.shape}')

Feature Scaling

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()

In [ ]:
scaler.fit(x_train)

In [ ]:
X_train=scaler.transform(x_train)
X_test =scaler.transform(x_test)

Model Creation

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [ ]:
model = Sequential()
model.add(Dense(units=8,activation='relu', kernel_initializer='he_normal',input_dim=X_train.shape[1] ))
model.add(Dense(units=8, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(units=8, activation='sigmoid', kernel_initializer='glorot_normal'))
model.add(Dense(units=1))

In [ ]:
model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, batch_size=70, epochs=150)

In [ ]:
y_pred=model.predict(X_test)

In [ ]:
predictions=y_pred > 0.5


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report,accuracy_score

In [ ]:
confusion_matrix(y_test, predictions)

In [ ]:
print(classification_report(y_test, predictions))

In [ ]:
accuracy_score(y_test, predictions)

Hyper parameter tuning

In [ ]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

In [ ]:
def create_model(layers, activation):
  model=Sequential()
  for i,layer in enumerate(layers):
    if i==0:
      model.add(Dense(units=layer, activation=activation,input_dim=X_train.shape[1]))
      model.add(Dropout(0.3))
    else:
      model.add(Dense(units=layer, activation=activation))
      model.add(Dropout(0.3))
    model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model
model = KerasClassifier(build_fn=create_model, verbose=0)

In [ ]:
model

In [ ]:
layers=[[20], [15,20], [15, 20, 25]]
activation=['sigmoid', 'relu']

In [ ]:
param_grid=dict(layers=layers,activation=activation, batch_size=[128,256], epochs=[75,100,125])

In [ ]:
param_grid

In [ ]:
grid=GridSearchCV(estimator=model, param_grid=param_grid )
my_grid=grid.fit(X_train,y_train)

In [ ]:
my_grid.best_score_

In [ ]:
my_grid.best_params_

Creating New Model

In [ ]:
model_new=Sequential()
model_new.add(Dense(units=15, activation='relu', kernel_initializer='he_uniform', input_dim=X_train.shape[1]))
model_new.add(Dense(units=20, activation='relu', kernel_initializer='he_uniform'))
model_new.add(Dense(units=25, activation='relu', kernel_initializer='he_uniform'))
model_new.add(Dense(units=1, activation='sigmoid', kernel_initializer='glorot_uniform'))

model_new.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model_new.fit(X_train,y_train, validation_split=0.2,epochs=100, batch_size=256)